# Private Eye - UK Land Ownership By Offshore Companies

*This notebook makes use of data acquired by Private Eye using an FOI request to the Land Registry revealing "all properties acquired by offshore companies from 1999 to 2014, showing the address, the offshore corporate owners (some have more than one) and, where available, the price paid", available via http://www.private-eye.co.uk/issue-1414/registry*.


*A couple of things to note about the data:*
- *it is not made available under an open license;*
- *the data file linked to by Private Eye is in an xlsb format. For use in this notebook, the data needs converting to an xlsx or csv format (for example by opening it in Microsoft Excel and then resaving it.*

This notebook provides a few illustrations of how to work with the data and the sorts of discussions we can have with it.

## Opening the data file

To being with, we can open the data file into a dataframe using the *pandas* data analysis package. This package provides a wide range of support for working with and analysing tabular datasets.

In [10]:
#Load in the pandas package
import pandas as pd

#Import the data - it's contained within a single sheet within the spreadsheet
df=pd.read_excel('overseas-company-dataset-december-2014.xlsx',encode='utf8')

## Preview the Data

We can preview the data in a couple of ways. For example, we can inspect the column headings:

In [15]:
df.columns

Index([u'Title number', u'Year', u'Date', u'District',
       u'Administrative county', u'Price paid', u'Price (text infill)',
       u'Proprietor', u'Country/territory', u'Variable', u'Address',
       u'Postcode', u'Tenure'],
      dtype='object')

We can see how many rows are contained in the datatable:

In [23]:
len(df)

100997

We can look at the first few rows of the dataset to get a feel for what's there:

In [14]:
#Preview the first three rows
df.head(3)

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
0,NGL676405,2014,2014-12-22,CITY OF WESTMINSTER,GREATER LONDON,2950000,2950000,EVERLINE PROPERTY II LIMITED,BRITISH VIRGIN ISLANDS,Proprietor(1),"Flat 6, Queensbury Court, 2 Hamilton Mews, Lon...",W1J 7HA,Leasehold
1,SY695373,2014,2014-12-19,RUNNYMEDE,SURREY,5150000,5150000,NEWINGTON CAUSEWAY INC,BRITISH VIRGIN ISLANDS,Proprietor(1),"land at Pound Road, Chertsey",NaN,Freehold
2,K600274,2014,2014-12-19,MAIDSTONE,KENT,2100000,2100000,EATON ASSETS UK LIMITED,JERSEY,Proprietor(1),"Priory Gate, Union Street, Maidstone",NaN,Freehold


We can get a summary (and count) of how many register entries are made in each territory, limiting the display to the top 10:

In [31]:
df['Country/territory'].value_counts()[:10]

BRITISH VIRGIN ISLANDS    22155
JERSEY                    20590
ISLE OF MAN               12061
GUERNSEY                  11536
UNABLE TO CONFIRM          3282
MAURITIUS                  2782
GIBRALTAR                  2657
LUXEMBOURG                 2513
PANAMA                     1963
EIRE                       1957
Name: Country/territory, dtype: int64

You might realise that it's simple enough to run a similar command to get counts or register entries made in a particular *Administrative county*:

In [33]:
df['Administrative county'].value_counts()[:10]

GREATER LONDON        43839
GREATER MANCHESTER     5283
WEST YORKSHIRE         2638
MERSEYSIDE             2574
WEST MIDLANDS          2451
SURREY                 2399
KENT                   1885
ESSEX                  1742
LANCASHIRE             1608
TYNE AND WEAR          1494
Name: Administrative county, dtype: int64

or *District*:

In [35]:
df['District'].value_counts()[:10]

CITY OF WESTMINSTER       13128
KENSINGTON AND CHELSEA     6769
CAMDEN                     2431
TOWER HAMLETS              1837
WANDSWORTH                 1834
MANCHESTER                 1625
LAMBETH                    1576
LEEDS                      1534
HAMMERSMITH AND FULHAM     1482
BARNET                     1351
Name: District, dtype: int64

or *Proprietor*:

In [36]:
df['Proprietor'].value_counts()[:10]

WALLACE PROPERTIES LIMITED                                        2255
PROFITABLE PLOTS PTE LTD                                          1206
SOLARFAIR INVESTMENTS LIMITED                                     1178
BNP PARIBAS SECURITIES SERVICES TRUST COMPANY (JERSEY) LIMITED     704
BNP PARIBAS SECURITIES SERVICES TRUST COMPANY LIMITED              670
WESTMINSTER BRIDGE HOLDINGS B.V.                                   481
STONEGATE PUB COMPANY LIMITED                                      458
RBC TRUSTEES (CI) LIMITED                                          454
ADRIATIC LAND 2 LIMITED                                            444
ANDERTON ESTATE INVESTMENTS LIMITED                                423
Name: Proprietor, dtype: int64

Hmm... is that an easy way of finding all the pubs owned by the Stonegate Pub Company (whose [website](http://www.stonegatepubs.com/) claims they currently operate *"over 660 pubs and bars across the UK"*?! (Actually, it looks as if there wesbite can be esily scraped to grab that data...)

Or properties that have possibly had solar panels installed, (thanks to Simon Perry of OnTheWight.com for that observation). In which case, do any of those properties look like they may have something in common...?

## Poking Around Particular Companies

Let's see if we can learn anything by eyeballing the *Address* field for properties associated with a particular company:

In [47]:
df[df['Proprietor']=='SOLARFAIR INVESTMENTS LIMITED']['Address'][:10]

11468    Airspace above 19 Stillman Close, Bristol, BS1...
11469    Airspace above 3 Barton Green, Barton Hill Roa...
11470    Airspace above 8 Cousins Close, Bristol, BS10 7XH
11471    Airspace above 15 Barton Green, Barton Hill Ro...
11472    Airspace above 4 Catherine Mead Street, Bristo...
11473    Airspace above 5 Catherine Mead Street, Bristo...
11474    Airspace above 129 Ashton Drive, Bristol, BS3 2PS
11475    Airspace above 72 Lawrence Weston Road, Bristo...
11476    Airspace above 409 Bishport Avenue, Bristol, B...
11477    Airspace above 17 Livale Road, Bettws, Newport...
Name: Address, dtype: object

In [48]:
df[df['Proprietor']=='SOLARFAIR INVESTMENTS LIMITED']['Tenure'].value_counts()

Leasehold    1178
Name: Tenure, dtype: int64

So - *"airspace above"*... It seems that many companies installing domestic solar panels on residential properties as part of a leasing agreement also took a lease out on the "air above" those properties [Loosemores Solicitors - Conveyancing and Solar Panels](http://www.loosemores.co.uk/solar-panels/).

Are any of the other highly represented companies engaging in similar leasing agreements?

In [55]:
df[df['Address'].str.contains('Airspace', na=False)]['Proprietor'].value_counts()[:3]

SOLARFAIR INVESTMENTS LIMITED    1071
AYOT LIMITED                      194
WARRAWEE INVESTMENTS LIMITED        7
Name: Proprietor, dtype: int64

*Ayot Limited* perhaps...?

A couple of other patterns we might look for when it comes to individual companies. First, are they concentrating by postcode (and might that indicate something?)

In [65]:
df[['Proprietor','Postcode']].groupby(['Proprietor','Postcode']).agg(len).sort_values(ascending=False)[:10]

Proprietor                           Postcode
WESTMINSTER BRIDGE HOLDINGS B.V.     SE1 7UT     481
CVI LEEDS WATERSIDE SARL             LS12 1DW     82
TABIT LIMITED                        M50 1DB      81
WINOCA LIMITED                       M3 6GF       81
CASHALSTONE DEVELOPMENTS LIMITED     E1 0HJ       79
LSP RI MOORE HOUSE LIMITED           SW1W 8DU     75
ANDERTON ESTATE INVESTMENTS LIMITED  WA11 9AP     68
LSP RI MOORE HOUSE LIMITED           SW1W 8DT     68
CUBIC SPV LIMITED                    NG1 5GD      67
SARUNAS PROPERTIES LIMITED           L8 0PY       63
dtype: int64

Let's see if we can (by eye) spot any commonalities in the listings for the first of those:

In [68]:
df[df['Proprietor'].str.startswith("WESTMINSTER BRIDGE")][:5]

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
16012,TGL332629,2013,2013-05-07,LAMBETH,GREATER LONDON,157500,157500,WESTMINSTER BRIDGE HOLDINGS BV,NETHERLANDS,Proprietor(1),"Flat 953, Park Plaza Westminster Bridge, 200 W...",SE1 7UT,Leasehold
36021,TGL349480,2011,2011-07-12,LAMBETH,GREATER LONDON,NaN,"£120,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1228, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36022,TGL349482,2011,2011-07-12,LAMBETH,GREATER LONDON,NaN,"£120,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1230, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36023,TGL349483,2011,2011-07-12,LAMBETH,GREATER LONDON,NaN,"£120,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1231, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36024,TGL349485,2011,2011-07-12,LAMBETH,GREATER LONDON,NaN,"£120,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1238, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold


So - different lots in a particular building, perhaps?

Looking at the title numbers, it might be worth quickly ordering on the *Title* column and eyeballing the result to see if there are runs of numbers with small gaps, and then perhaps looking up the missing items just to see what's there...

In [79]:
df[df['Proprietor'].str.startswith("WESTMINSTER BRIDGE")].sort_values('Title number')[13:18]

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
36722,TGL348519,2011,2011-06-28,LAMBETH,GREATER LONDON,NaN,"£165,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1411, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36723,TGL348522,2011,2011-06-28,LAMBETH,GREATER LONDON,NaN,"£165,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1413, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36710,TGL348523,2011,2011-06-28,LAMBETH,GREATER LONDON,NaN,"£135,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1417, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36673,TGL348524,2011,2011-06-28,LAMBETH,GREATER LONDON,NaN,"£637,500 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1418, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold
36675,TGL348526,2011,2011-06-28,LAMBETH,GREATER LONDON,NaN,"£120,000 plus VAT",WESTMINSTER BRIDGE HOLDINGS B.V.,NETHERLANDS,Proprietor(1),"Room 1002, Park Plaza Westminster Bridge, 200 ...",SE1 7UT,Leasehold


Another way of trying to find patterns of behaviour for a particular proprietor is to group by *Proprietor* and *Date* (this sort of thing can be interesting when analysing company registrations when looking at bulk company registration data...):

In [69]:
df[['Proprietor','Date']].groupby(['Proprietor','Date']).agg(len).sort_values(ascending=False)[:10]

Proprietor                           Date      
WALLACE PROPERTIES LIMITED           2012-05-25    2205
LINK HOLDINGS (GIBRALTAR) LIMITED    2006-07-24     348
SOLARFAIR INVESTMENTS LIMITED        2012-12-10     322
CVI LEEDS WATERSIDE SARL             2013-10-07     255
STONEGATE PUB COMPANY LIMITED        2010-12-01     255
ANDERTON ESTATE INVESTMENTS LIMITED  2009-02-11     255
AMATEX LIMITED                       2012-07-16     234
NEWRIVER TRUSTEE 7 LIMITED           2014-02-06     229
NEWRIVER TRUSTEE 8 LIMITED           2014-02-06     229
AYOT LIMITED                         2013-03-27     197
dtype: int64

In [72]:
df[df['Proprietor'].str.startswith("WALLACE PROPERTIES")][:5]

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
14856,MM21129,2013,2013-06-28,SOLIHULL,WEST MIDLANDS,NaN,NaN,WALLACE PROPERTIES LIMITED,MAURITIUS,Proprietor(1),"Land lying to the East of Langfield Road, Know...",NaN,Freehold
17910,SF277301,2013,2013-03-19,TAMWORTH,STAFFORDSHIRE,NaN,NaN,WALLACE PROPERTIES LIMITED,MAURITIUS,Proprietor(1),"15, Glyndebourne, Tamworth, B79 7UD",B79 7UD,Freehold
19140,WYK249388,2013,2013-01-30,LEEDS,WEST YORKSHIRE,NaN,NaN,WALLACE PROPERTIES LIMITED,MAURITIUS,Proprietor(1),"land adjoining 9 Alma Road, Headingley",NaN,Freehold
21506,MAN201650,2012,2012-11-27,BURY,GREATER MANCHESTER,NaN,NaN,WALLACE PROPERTIES LIMITED,MAURITIUS,Proprietor(1),Land on the north west side of Milltown Street...,NaN,Freehold
21566,MM12372,2012,2012-11-23,BIRMINGHAM,WEST MIDLANDS,NaN,NaN,WALLACE PROPERTIES LIMITED,MAURITIUS,Proprietor(1),"85, Pipers Green, Birmingham, B28 0NU",B28 0NU,Freehold


## Filter the Data to a Particular Locale

One of the other ways of slicing the data is to focus on a particular locale. For example, are there any patterns of ownership by offshore companies on the Isle of Wight?

Let's filter down by *Administrative County*:

In [82]:
dfxiw=df[df['Administrative county'].str.contains('WIGHT')]

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
15651,IW78225,2013,2013-05-22,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,SALCON POWER (HK) LIMITED,HONG KONG,Proprietor(1),"4, The Ridgeway, Sandown, PO36 9DU",PO36 9DU,Leasehold
15652,IW78226,2013,2013-05-22,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,SALCON POWER (HK) LIMITED,HONG KONG,Proprietor(1),"30, St Edmunds Walk, Wootton Bridge, Ryde, PO3...",PO33 4JB,Leasehold
15653,IW78227,2013,2013-05-22,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,SALCON POWER (HK) LIMITED,HONG KONG,Proprietor(1),"58, St Edmunds Walk, Wootton Bridge, Ryde, PO3...",PO33 4JB,Leasehold


For convenience, we might want to save this local data into its own file.

In [174]:
dfxiw.to_csv('IWoverseasOwnedPropeties.csv',index=False,encoding='utf-8')

We can now run some of the same sorts of query as we ran before but on this smaller, locally based dataset.

In [100]:
dfxiw['Proprietor'].value_counts()[:10]

AYOT LIMITED                           31
TIFFIELD LIMITED                        9
KLEINWORT BENSON (GUERNSEY) LIMITED     7
REDWOOD DEVELOPMENTS LIMITED            7
BORROWDALE NOMINEES LIMITED             7
GUITON GROUP LIMITED                    6
XYZ LIMITED                             6
SHAY PROPERTY INVESTMENTS S.AR.L        5
HATTON CONSULTANTS LIMITED              4
RBC TRUSTEES (CI) LIMITED               4
Name: Proprietor, dtype: int64

In [103]:
dfxiw[['Postcode', 'Proprietor']].groupby(['Postcode', 'Proprietor']).agg(len).sort_values(ascending=False)[:15]

Postcode  Proprietor                             
PO30 1JW  TIFFIELD LIMITED                           4
PO33 1BX  HORIZON TRUSTEES (JERSEY) LIMITED          3
PO30 1JL  TIFFIELD LIMITED                           3
PO30 5LH  SHAY PROPERTY INVESTMENTS S.AR.L           3
PO36 9PH  HIGH TIDE (CI) LTD                         2
PO30 2EL  RBC TRUSTEES (CI) LIMITED                  2
PO30 3AJ  AYOT LIMITED                               2
PO30 3HG  FORTRESS INTERNATIONAL SERVICES LIMITED    2
PO30 5XB  HATTON CONSULTANTS LIMITED                 2
PO31 7DQ  GIANNA DEVELOPMENTS LIMITED                2
PO31 7XF  DERRY BEG (JERSEY) LIMITED                 2
PO33 4JB  SALCON POWER (HK) LIMITED                  2
PO33 4LN  AYOT LIMITED                               2
PO41 0PN  EDRA SECURITIES LIMITED                    2
PO38 3HH  XYZ LIMITED                                2
dtype: int64

We can probe a little deeper on specific companies who've got the title to several properties in the same postcode area: 

In [102]:
dfxiw[(dfxiw['Postcode']=='PO30 1JW') & (dfxiw['Proprietor']=='TIFFIELD LIMITED')]

,Title number,Year,Date,District,Administrative county,Price paid,Price (text infill),Proprietor,Country/territory,Variable,Address,Postcode,Tenure
54636,IW41343,2008,2008-12-12,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,TIFFIELD LIMITED,BRITISH VIRGIN ISLANDS,Proprietor(1),"1a, Pyle Street, Newport, PO30 1JW",PO30 1JW,Freehold
54641,IW48084,2008,2008-12-12,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,TIFFIELD LIMITED,BRITISH VIRGIN ISLANDS,Proprietor(1),"2, 3 and 4 Pyle Street, Newport, PO30 1JW",PO30 1JW,Freehold
54642,IW48503,2008,2008-12-12,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,TIFFIELD LIMITED,BRITISH VIRGIN ISLANDS,Proprietor(1),"1b, Pyle Street, Newport, PO30 1JW",PO30 1JW,Freehold
54643,IW6401,2008,2008-12-12,ISLE OF WIGHT,ISLE OF WIGHT,NaN,NaN,TIFFIELD LIMITED,BRITISH VIRGIN ISLANDS,Proprietor(1),"1, Pyle Street, Newport, PO30 1JW",PO30 1JW,Freehold


So is someone buying up Pyle Street in Newport?

## Some Simple Mapping

One of the natural ways of looking at geographical data is on a map. To do this we need to do a couple of things:

- *geocode* the data to get latitude and longitude data for each address; a crude way of doing this is by postcode;
- plot the points on a map.

To begin with, let's geocode the postcodes. There are several online services that help us do this - I'm going to use *postcodes.io*. For convenience, let's define a function that will geocode a list of postcodes for us.

In [124]:
import time
import json
import requests

#Define a function to geocode a list of postcodes
def getPC(pclist):
    #Get just unique postcodes
    pclist=set(pclist)
    pclookup={}
    for pc in pclist:
        #Play nicely withthe API - put a delay in between each request
        time.sleep(1)
        #print(pc)
        url='http://postcodes.io/postcodes/{pc}'.format(pc=pc.replace(' ',''))

        jdata=json.loads(requests.get(url).content)

        if 'result' in jdata:
            pclookup[pc]={'data':jdata['result'],'lat':jdata['result']['latitude'], 'lng':jdata['result']['longitude']}

    return pclookup

The `getPC()` function will try to find the latitude and longitude for us, and pop it into a dictionary keyed by the postcode.

In [ ]:
#Geocode the Isle of Wight postcodes, limiting the lookup to unique, non-zero postcodes
dfxiwpc=getPC(dfxiw['Postcode'].dropna().unique())

We can now create another, possiblly smaller, local area dataframe that contains just rows that we have valid co-ordinates for, and add in those co-ordinates.

In [ ]:
#Filter the IW data to just rows with a postcode
dfxiw2=dfxiw[pd.notnull(dfxiw['Postcode'])]
#And then filter further to just rows with a postcode we managed to geocode
dfxiw2=dfxiw2[dfxiw['Postcode'].isin(dfxiwpc)]
#Now add in the lat/long data for those rows
dfxiw2['lat']=dfxiw2['Postcode'].apply(lambda x: dfxiwpc[x]['lat'])
dfxiw2['lon']=dfxiw2['Postcode'].apply(lambda x: dfxiwpc[x]['lng'])

With the rows annotated with geographical co-ordinates information, we can now plot a map showing approximately where the foreign owned titles apply to.

To make things a little easier, we'll use the`folium` mapping library, and also define a simple function (`mapper()`) to add markers to the map.

In [171]:
#Depending on your Python environment, you may need to run:
#!pip install folium
#or:
#!pip3 install folium
import folium

def mapper(folium_map,df):
    ''' Add markers contained in df to folium_map  '''
    for row in df[ df['lon'].notnull() ].iterrows():
        #Use a simple circle marker
        #Add a simple popup legend to each one
        #Note that the circle marker doesn't let us distinguish between overlapping markers
        folium_map.add_children(folium.features.CircleMarker([row[1]['lat'], row[1]['lon']],
                                                     radius=10, color='red',fill_color='red',
                                                      popup='{}: {}'.format(row[1]['Proprietor'], row[1]['Address'])))
    return folium_map

Now we can render the map:

In [172]:
#Set the mid_point of the map as the mean lat/long
#(A slightly better way of doing this is to find the centroid of the points)
longm=dfxiw2[ dfxiw2['lon'].notnull() ]['lon'].mean()
latm=dfxiw2[ dfxiw2['lat'].notnull() ]['lat'].mean()

#Create the map
pricesmap = folium.Map(location=[latm, longm], zoom_start=11)

#Add the markers
pricesmap=mapper(pricesmap,dfxiw2)

#Render the map
pricesmap

## Taking Things Further

We might me able to take the investigation further by looking for evidence of planning applications made against particular properties (which in turn might turn up agent names), for example via the [Isle of Wight Council Planning Application website](https://www.iwight.com/planning/planAppSearchHistory.aspx), or looking up to see whether there are similarly named companies on [OpenCorporates](http://opencorporates.com) or [Companies House](https://beta.companieshouse.gov.uk/search/).